<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/Exp_13_LightGBM_Improvement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Improvements that I tried on LightGBM:

*   Word2vec: For vectorization. Word2vec suprisingly didn't perform that well. It still couldn't outperform Tf-Idf.

*   Custom features: I added custom features like comment length, word count, average word length, unique word count, lexical diversity, Part of Speech tags count and Proportion of POS tags. There were small improvements in '0_precision' and '1_recall' but -1_recall and other reduced a bit. So, I would still go with the initial LightGBM model without custom features

In the next experiment:-
*   Stacking: Next, I will try stacking. I will use 3 base learners: XGBoost, LightGBM and SVM. And for the meta learner model, I will use Logistic Regression.

*   BERT-based analysis





# Custom Features

In [1]:
!pip install mlflow boto3 awscli optuna lightgbm imbalanced-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 105.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85

In [2]:

import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())


{'ResponseMetadata': {'RequestId': '95T6067395SXGNKS', 'HostId': 'E7h5c1IUYpA29J1OtpjXUNu77MgaehlhIW2lUOTq/bfwc1/zorkMHqMgKvOilHhk1/SU6+ZIwaw=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'E7h5c1IUYpA29J1OtpjXUNu77MgaehlhIW2lUOTq/bfwc1/zorkMHqMgKvOilHhk1/SU6+ZIwaw=', 'x-amz-request-id': '95T6067395SXGNKS', 'date': 'Sun, 19 Oct 2025 00:48:33 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'adityasahusomu02', 'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [4]:

import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [5]:
mlflow.set_experiment("exp 6 -LightGBM with HP Tuning")


<Experiment: artifact_location='s3://project1-mlflow-bucket/131227091685296502', creation_time=1760614823705, experiment_id='131227091685296502', last_update_time=1760614823705, lifecycle_stage='active', name='exp 6 -LightGBM with HP Tuning', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

In [7]:
url = "https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv"
df = pd.read_csv(url).dropna(subset = ['clean_comment'])
df.shape

(36662, 2)

In [8]:
df.head(2)

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1


In [9]:
X = df['clean_comment']
y = df['category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
nlp = spacy.load('en_core_web_sm')         # small English Language Model

In [18]:
def extract_custom_features(text):
  doc = nlp(text)
  word_list = [token.text for token in doc]

  # feature_1:comment length (# charecters)
  comment_length = len(text)

  # feature_2: word count
  word_count = len(word_list)

  # feature_3: average word length
  avg_word_length = sum(len(word) for word in word_list)/word_count if word_count>0 else 0

  # feature_4: Unique word count
  unique_word_count = len(set(word_list))

  # feature_5: Lexical diversity
  lexical_diversity = unique_word_count/word_count if word_count>0 else 0

  # feature_6: POS tags count
  pos_count = len([token.pos_ for token in doc])

  # feature_7: Proportion of POS tags
  pos_tags = [token.pos_ for token in doc]
  pos_proportion = {tag: pos_tags.count(tag) / word_count for tag in set(pos_tags)} if word_count > 0 else {}

  return {
      'comment_length': comment_length,
      'word_count': word_count,
      'avg_word_length': avg_word_length,
      'unique_word_count': unique_word_count,
      'lexical_diversity': lexical_diversity,
      'pos_count': pos_count,
      **pos_proportion}  # Flattening the POS proportions



In [19]:
custom_features_train = pd.DataFrame([extract_custom_features(text) for text in X_train])
custom_features_test = pd.DataFrame(extract_custom_features(text) for text in X_test)

In [20]:
custom_features_train.head(3)

,comment_length,word_count,avg_word_length,unique_word_count,lexical_diversity,pos_count,VERB,AUX,NOUN,ADP,...,ADJ,PART,NUM,INTJ,SCONJ,X,CCONJ,DET,PUNCT,SYM
0,51,7,6.428571,7,1.0,7,0.428571,0.142857,0.142857,0.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36,6,5.166667,6,1.0,6,0.333333,NaN,0.333333,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64,9,6.222222,9,1.0,9,0.222222,NaN,0.444444,NaN,...,0.222222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
custom_features_train.isnull().sum()

,0
comment_length,0
word_count,0
avg_word_length,0
unique_word_count,0
lexical_diversity,0
pos_count,0
VERB,5797
AUX,22496
NOUN,3217
ADP,24100


In [22]:
custom_features_train.fillna(0, inplace=True)
custom_features_test.fillna(0, inplace=True)

In [24]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_features=10000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [30]:
X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf.get_feature_names_out())

In [33]:
X_train_combined = pd.concat([X_train_tfidf_df.reset_index(drop=True), custom_features_train.reset_index(drop =True)], axis=1)
X_test_combined = pd.concat([X_test_tfidf_df.reset_index(drop=True), custom_features_test.reset_index(drop =True)], axis=1)

In [35]:
X_train_combined.head()      # 10000 max_features columns + 23 custom_features columns

,000,000 000,000 crore,000 rupee,100,100 crore,100 time,100 year,1000,1000 note,...,ADJ,PART,NUM,INTJ,SCONJ,X,CCONJ,DET,PUNCT,SYM
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.222222,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.200000,0.066667,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:

from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna

In [41]:
ngram_range = (1, 3)
max_features = 10000
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


def objective_improved_lightgbm(trial):                                 # Defining the Objective function of Optuna
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 30)

    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           objective='multiclass',
                           num_class=3,
                           metric="multi_logloss",
                           is_unbalance= True,
                           class_weight= "balanced",
                           reg_alpha= 0.1,  # L1 reg
                           reg_lambda= 0.1,  # L2 reg
                           random_state=42)
    return accuracy_score(y_test, model.fit(X_train_combined, y_train).predict(X_test_combined))


# Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_improved_lightgbm, n_trials=50)

    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    log_mlflow("Improved_LightGBM-custom-features", best_model, X_train_combined, X_test_combined, y_train, y_test)

run_optuna_experiment()

[I 2025-10-19 03:55:28,385] A new study created in memory with name: no-name-42e8f1b2-083a-4317-b810-ba8909a85dd0


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.656836 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 136999
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 4461
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

[I 2025-10-19 03:57:27,414] Trial 0 finished with value: 0.807854902495568 and parameters: {'n_estimators': 396, 'learning_rate': 0.06484378400115597, 'max_depth': 11}. Best is trial 0 with value: 0.807854902495568.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 3.081179 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 136999
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 4461
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

[I 2025-10-19 04:00:29,135] Trial 1 finished with value: 0.8396290740488204 and parameters: {'n_estimators': 383, 'learning_rate': 0.06613665664209907, 'max_depth': 24}. Best is trial 1 with value: 0.8396290740488204.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.855946 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 136999
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 4461
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2025-10-19 05:41:33,100] Trial 49 finished with value: 0.6033001500068185 and parameters: {'n_estimators': 375, 'learning_rate': 0.0014367175624828742, 'max_depth': 25}. Best is trial 34 with value: 0.8442656484385654.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.479480 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 136999
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 4461
[LightGBM] [Info] Start training from score -1.491356
[LightGBM] [Info] Start training from score -1.062681
[LightGBM] [Info] Start training from score -0.845356
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


2025/10/19 05:43:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/19 05:43:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Improved_LightGBM-custom-features_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/131227091685296502/runs/0e70d5f8c8ae4a34bcdbbac98fdcbb2a
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/131227091685296502
